In [1]:
import numpy as np
import tensorflow as tf

In [ ]:
X=np.load('/home/anirudh_seth/X1.npy',allow_pickle=False)
Y=np.load('/home/anirudh_seth/Y1.npy',allow_pickle=False)
X=np.asarray(X).transpose(0,2,1)
Y=np.asarray(Y).transpose(0,2,1)

In [ ]:
# embeddingDim=10
# timesteps = 154
# n_features = 13

In [ ]:
class MyModel(Model):
  def __init__(self):
    embeddingDim=10
    timesteps = 154
    n_features = 13
    super(MyModel, self).__init__()
    self.l1=tf.keras.layers.LSTM(units=embeddingDim,activation='relu',input_shape=(timesteps,n_features),return_sequences=True)
    self.l2=tf.keras.layers.LSTM(units=5,activation='relu',return_sequences=False,input_shape=(timesteps,n_features))
    self.l3=tf.keras.layers.RepeatVector(timesteps)
    self.l4=tf.keras.layers.LSTM(5,activation='relu',return_sequences=True)
    self.l5=tf.keras.layers.LSTM(embeddingDim,activation='relu',return_sequences=True)
    self.l6=tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))


  def call(self, x):
    x = self.l1(x)
    x = self.l2(x)
    x = self.l3(x)
    x = self.l4(x)
    x = self.l5(x)
    return self.d6(x)

# Create an instance of the model
model = MyModel()

In [ ]:
loss_object = tf.keras.losses.mean_squared_error
optimizer = tf.keras.optimizers.Adam()

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

In [ ]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [ ]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch + 1,
                        train_loss.result(),
                        train_accuracy.result() * 100,
                        test_loss.result(),
                        test_accuracy.result() * 100))